In [31]:
import pandas as pd
import pickle
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import Imputer
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.externals import joblib
from xgboost import XGBRegressor
from sklearn.gaussian_process import GaussianProcess
from sklearn.feature_selection import f_regression

In [2]:
train = pd.read_csv('data/ecs171train.csv')
# data imputation
fill_NaN = Imputer(missing_values=np.nan, strategy='mean', axis=0)
test = pd.DataFrame(fill_NaN.fit_transform(train))
scaler = StandardScaler()
test = pd.DataFrame(scaler.fit_transform(test))
test.columns = train.columns
test.index = train.index

In [3]:
train

,Unnamed: 0,id,f1,f2,f3,f4,f5,f6,f7,f8,...,f770,f771,f772,f773,f774,f775,f776,f777,f778,loss
0,0,57339,129,9,0.244479,1300,16,83366,4598.0,4024.0,...,2,0.97,-0.67,0.47,0.0444,3.8046,1,0,394,2
1,1,16253,144,9,0.924478,3600,3,83292,623.0,822.0,...,8,0.91,-0.43,0.20,0.1589,-1.0681,1,1,67,0
2,2,44671,159,7,0.336656,5500,4,76635,3136.0,4014.0,...,17,6.19,-4.72,3.77,0.2959,0.6115,0,1,2,0
3,3,20253,117,8,0.969409,1300,4,79371,198.0,4892.0,...,2,0.06,-0.02,0.00,0.1199,-1.1551,1,0,46,0
4,4,101418,140,6,0.470937,2600,7,84020,NaN,8996.0,...,6,2.79,-2.09,1.62,0.1838,0.6018,0,0,2,0
5,5,44816,127,10,0.044531,1300,16,85131,7408.0,3710.0,...,64,22.03,-15.94,12.12,0.2699,0.6795,0,0,393,0
6,6,100356,130,7,0.244225,1300,4,77014,6548.0,10073.0,...,15,4.92,-3.40,2.42,0.2341,0.5427,0,0,49,0
7,7,105060,137,8,0.154747,2200,4,13317,1669.0,4336.0,...,13,4.70,-3.34,2.49,0.2359,0.7034,0,0,54,0
8,8,41601,152,9,0.049318,3600,13,80120,1487.0,1059.0,...,3,0.09,-0.02,0.01,0.0739,-0.8627,1,1,27,0
9,9,19919,159,7,0.157096,3700,4,9482,7398.0,3403.0,...,11,4.80,-3.42,2.51,0.1671,0.9216,0,1,39,0


In [4]:
test['loss'] = train['loss']
test = test[test.columns.values[2:]]

In [5]:
test

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f770,f771,f772,f773,f774,f775,f776,f777,f778,loss
0,-0.382530,0.445183,-0.887742,-0.985801,1.673296,0.990708,6.384724e-01,0.712434,-0.512582,-0.408762,...,-0.827754,-0.738853,0.737430,-0.740789,-2.583372,3.633460,1.486560,-0.692782,0.725807,2
1,0.636094,0.445183,1.470515,0.656789,-0.846063,0.988634,-9.251128e-01,-0.714153,0.714351,0.670968,...,-0.504313,-0.748065,0.787093,-0.812399,-1.020525,-1.038089,1.486560,1.443456,-0.367132,0
2,1.654719,-0.741023,-0.568070,2.013712,-0.652266,0.802070,6.338773e-02,0.707979,1.295986,1.599701,...,-0.019152,0.062578,-0.100637,0.134444,0.849433,0.572175,-0.672694,1.443456,-0.584383,0
3,-1.197430,-0.147920,1.626338,-0.985801,-0.652266,0.878747,-1.092289e+00,1.099154,-1.181752,-1.195462,...,-0.827754,-0.878566,0.871934,-0.865443,-1.552848,-1.121497,1.486560,-0.692782,-0.437321,0
4,0.364461,-1.334125,-0.102379,-0.057381,-0.070875,1.009037,-1.788770e-16,2.927609,-0.097335,0.292442,...,-0.612127,-0.459427,0.443589,-0.435784,-0.680656,0.562876,-0.672694,-0.692782,-0.584383,0
5,-0.518347,1.038286,-1.581170,-0.985801,1.673296,1.040173,1.743799e+00,0.572538,-0.731057,-0.570101,...,2.514467,2.494508,-2.422392,2.349048,0.494550,0.637368,-0.672694,-0.692782,0.722465,0
6,-0.314622,-0.741023,-0.888624,-0.985801,-0.652266,0.812691,1.405514e+00,3.407445,-0.281809,-0.298445,...,-0.126966,-0.132406,0.172511,-0.223606,0.005905,0.506215,-0.672694,-0.692782,-0.427294,0
7,0.160736,-0.147920,-1.198937,-0.343049,-0.652266,-0.972438,-5.136637e-01,0.851439,0.083522,0.172472,...,-0.234779,-0.166183,0.184926,-0.205041,0.030473,0.660281,-0.672694,-0.692782,-0.410582,0
8,1.179361,0.445183,-1.564569,0.656789,1.091906,0.899738,-5.852543e-01,-0.608562,1.413905,1.214281,...,-0.773847,-0.873960,0.871934,-0.862791,-2.180717,-0.841168,1.486560,1.443456,-0.500825,0
9,1.654719,-0.741023,-1.190792,0.728206,-0.652266,-1.079915,1.739866e+00,0.435760,1.809620,1.703813,...,-0.342593,-0.150830,0.168372,-0.199736,-0.908600,0.869474,-0.672694,1.443456,-0.460717,0


In [28]:
#classify_feature = ['diff_f528_f527', 'diff_f274_f527', 'f2', 'f271', 'diff_f8_f582', 'f222', 'diff_f274_f528', 'f332', 'f576', 'f670']
classify_feature = ['diff_f528_f527', 'diff_f274_f527', 'f2', 'f271', 'diff_f8_f582', 'f222', 'diff_f274_f528', 'f332']
features = [467, 603, 530, 666, 550, 280, 279, 398, 312, 321, 320, 313, 403, 667, 756, 758, 665, 767, 249, 288, 291, 292, 293, 294, 217, 600, 290, 402, 757, 419, 333, 413, 219, 426, 412, 520, 611, 766, 527, 315, 316, 317, 318, 289, 654, 509, 517, 586, 583, 314, 746, 656, 585, 690, 612, 414, 214, 296, 658, 587, 764, 588, 737, 215, 653, 218, 424, 745, 359, 283, 297, 657, 763, 250, 110, 239, 591, 593, 67, 213, 744, 592, 109, 68, 659, 514, 63, 212, 762, 515, 111, 513, 45, 66, 342, 537, 547, 557, 567, 576]
feature_pair_sub_list = [[520, 521], [271, 521], [271, 520], [67, 466], [623, 664], [7, 536], [66, 529], [531, 532], [561, 562], [248, 602], [570, 571], [218, 766], [64, 765], [208, 590], [423, 660], [312, 463], [290, 592], [621, 755], [52, 311], [65, 422], [350, 656], [278, 420], [320, 633], [507, 761], [0, 341], [139, 665], [10, 724], [53, 319], [367, 698], [279, 421], [9, 358], [48, 287], [375, 653], [397, 728], [197, 666], [38, 295], [402, 758], [403, 757], [549, 584], [238, 258], [296, 526], [586, 607], [291, 591], [62, 289], [16, 288], [581, 589], [8, 380], [655, 683], [58, 582]]
loss_feature = []
feature_pair_plus_list = [[466, 529], [664, 759], [602, 766], [64, 665], [279, 590], [397, 592], [311, 621], [248, 755], [660, 768], [218, 666], [65, 278], [549, 607], [16, 402], [53, 757], [463, 526], [197, 312], [507, 762], [320, 619], [367, 380], [10, 350], [62, 401], [52, 756], [610, 633], [0, 656], [319, 758], [38, 50], [288, 296], [67, 584], [48, 611], [422, 724], [249, 591], [58, 686], [287, 295], [341, 589], [208, 728], [66, 508], [44, 605], [4, 358], [9, 695]]
feature_pair_mul_list = [[466, 529], [621, 664], [159, 626], [599, 602], [213, 607], [209, 218], [433, 463], [16, 665], [619, 766], [158, 625], [558, 605], [64, 248], [402, 660], [583, 606], [53, 279], [595, 596], [367, 590], [592, 633], [52, 278], [65, 350], [10, 38], [526, 644], [42, 397], [23, 666], [401, 758], [67, 73], [54, 589], [507, 549], [358, 591], [423, 610], [250, 312], [311, 755], [66, 353], [611, 732], [645, 765], [1, 320], [88, 341], [319, 757], [286, 296], [375, 403], [48, 509], [203, 581], [422, 655], [62, 87], [283, 622], [627, 724], [258, 686], [168, 268], [0, 197], [282, 646], [420, 656]]
feature_pair_divide_list = [[712, 664], [466, 726], [463, 6], [539, 529], [602, 507], [660, 653], [64, 208], [766, 630], [590, 197], [592, 0], [100, 145], [218, 3], [65, 9], [526, 525], [607, 643], [410, 647], [610, 144], [420, 278], [311, 10], [1, 279], [468, 462], [472, 444], [248, 652], [665, 510], [583, 143], [621, 59], [656, 52], [397, 509], [196, 700], [403, 401], [655, 54], [350, 518], [312, 672], [619, 615], [591, 187], [675, 671], [666, 157], [341, 8], [367, 217], [445, 402], [67, 637], [375, 654], [379, 716], [754, 549], [320, 517], [599, 596], [589, 267], [673, 718], [422, 400], [620, 386], [680, 719], [755, 28], [765, 383], [674, 721], [358, 717], [676, 720], [66, 167], [633, 347], [681, 71]]
feature_pair_sub_mul_list=[[271, 521, 1], [520, 521, 622], [7, 536, 218], [656, 664, 619], [664, 712, 621], [664, 719, 278], [664, 731, 401], [67, 466, 529], [466, 726, 65], [466, 754, 591], [664, 671, 402], [71, 664, 403], [342, 664, 280], [351, 664, 421], [623, 664, 666], [359, 664, 440], [368, 664, 510], [376, 664, 279], [414, 664, 420], [466, 539, 589], [424, 664, 665], [33, 664, 397], [466, 559, 10], [405, 664, 422], [664, 760, 439], [39, 664, 509], [55, 664, 620], [49, 664, 672], [66, 466, 549], [634, 664, 141], [267, 466, 433], [664, 703, 328], [60, 466, 64], [386, 664, 399], [38, 466, 633], [128, 466, 590], [615, 664, 511], [44, 466, 58], [107, 466, 157], [62, 466, 48], [7, 546, 248], [314, 507, 466], [531, 532, 209], [59, 664, 70], [126, 466, 367], [45, 664, 197], [246, 466, 118], [7, 556, 607], [117, 466, 592], [265, 466, 350], [97, 466, 156], [664, 702, 88], [7, 566, 258], [53, 466, 52], [215, 766, 466], [466, 584, 108], [236, 466, 72], [116, 466, 358], [61, 466, 56], [664, 704, 87], [311, 466, 28], [176, 466, 206], [29, 466, 631], [255, 466, 341], [247, 466, 0], [260, 466, 245], [664, 708, 98], [664, 696, 447], [664, 701, 601], [256, 466, 337], [664, 751, 431], [43, 466, 42], [46, 466, 36], [466, 602, 438], [40, 466, 717], [106, 466, 226], [319, 466, 758], [664, 697, 432], [410, 466, 686], [466, 741, 8], [250, 466, 235], [664, 693, 74], [86, 466, 630], [664, 752, 166], [18, 466, 632], [561, 562, 64], [313, 466, 54], [466, 746, 210], [466, 635, 9], [19, 466, 57], [63, 466, 517], [11, 466, 37], [237, 466, 216], [14, 466, 365], [664, 738, 127], [12, 466, 506], [96, 466, 338], [664, 709, 167], [326, 466, 764], [121, 466, 382], [47, 466, 15], [320, 466, 16], [664, 747, 227], [257, 466, 659], [186, 466, 139], [77, 529, 463], [321, 466, 375], [177, 466, 147], [187, 466, 217], [185, 466, 195], [294, 466, 660], [312, 466, 23], [664, 698, 225], [7, 575, 605], [266, 466, 650], [318, 466, 651], [664, 710, 599], [664, 739, 240], [17, 466, 348], [466, 582, 276], [664, 700, 441], [664, 694, 230], [404, 664, 393], [664, 748, 515], [570, 571, 64], [466, 767, 380], [287, 466, 356], [466, 706, 196], [295, 466, 339], [664, 740, 670], [664, 711, 627], [111, 466, 78], [50, 466, 654], [445, 466, 146], [448, 602, 768], [664, 699, 753], [664, 749, 277], [296, 466, 723], [406, 664, 394], [664, 695, 518], [188, 466, 190], [466, 587, 735], [466, 750, 285], [286, 466, 759], [664, 742, 220], [289, 466, 643], [165, 466, 366], [407, 664, 392], [155, 466, 21], [288, 466, 349], [109, 466, 175], [664, 743, 661], [664, 737, 207], [30, 607, 213], [664, 761, 3], [259, 466, 249], [178, 466, 373], [81, 466, 652], [322, 664, 101], [323, 664, 137], [238, 466, 27], [324, 664, 428], [325, 664, 91], [180, 466, 25], [68, 466, 653], [430, 466, 646], [79, 466, 357], [140, 664, 396], [327, 602, 745], [302, 466, 340], [31, 607, 558], [315, 664, 644], [32, 607, 214], [34, 607, 241], [35, 607, 242], [607, 668, 243], [607, 690, 244], [607, 691, 551], [607, 692, 552], [664, 730, 669], [310, 602, 655], [408, 664, 395], [297, 466, 26], [2, 602, 744], [526, 602, 757], [100, 602, 755], [160, 466, 24], [610, 766, 208], [189, 466, 170], [179, 466, 150], [316, 664, 138], [586, 607, 624], [317, 664, 614], [119, 602, 763], [526, 765, 645], [149, 466, 73], [99, 602, 756], [229, 602, 762], [110, 466, 400], [370, 664, 5], [602, 736, 732], [455, 463, 612], [664, 667, 270], [120, 602, 682], [466, 502, 595], [148, 466, 374], [228, 602, 681], [305, 602, 718], [611, 766, 516], [423, 660, 4], [89, 602, 369], [219, 602, 678], [239, 466, 715], [472, 602, 657], [168, 766, 353], [80, 466, 385], [158, 466, 22], [303, 602, 679], [90, 602, 720], [169, 602, 647], [664, 724, 362], [281, 660, 347], [159, 602, 680], [290, 664, 398], [492, 602, 683], [291, 664, 613], [292, 664, 20], [583, 602, 721], [293, 664, 384], [298, 664, 629], [508, 602, 675], [602, 606, 299], [602, 603, 300], [585, 602, 301], [463, 482, 381], [602, 714, 282], [304, 602, 360], [602, 713, 676], [588, 602, 673], [664, 728, 383], [568, 660, 538], [425, 660, 372], [221, 660, 222], [223, 660, 224], [572, 660, 577], [541, 660, 533], [542, 660, 251], [231, 660, 252], [232, 660, 253], [233, 660, 254], [234, 660, 261], [594, 602, 677], [593, 602, 722], [64, 364, 268], [306, 664, 355], [663, 664, 618], [563, 660, 553], [409, 466, 307], [450, 660, 13], [308, 664, 389], [345, 664, 729], [548, 660, 262], [474, 660, 346], [468, 660, 442], [82, 660, 628], [92, 660, 444], [309, 664, 637], [449, 660, 51], [93, 660, 462], [94, 660, 471], [487, 660, 596], [83, 660, 443], [486, 660, 598], [95, 660, 436], [84, 660, 597], [85, 660, 6], [608, 664, 275], [477, 660, 361], [484, 660, 454], [478, 660, 481], [496, 660, 491], [497, 660, 501], [102, 660, 263], [103, 660, 264], [104, 660, 75], [105, 660, 434], [724, 727, 642], [476, 660, 636], [112, 660, 354], [457, 660, 498], [113, 660, 363]]
feature_pair_sub_list_sf=[[271, 521], [520, 521], [271, 520], [592, 608], [238, 248], [186, 766], [60, 311], [176, 218], [9, 397], [248, 750], [90, 529], [396, 766], [660, 724], [108, 507], [107, 660], [65, 320], [577, 660], [271, 521], [10, 455], [64, 126], [56, 312], [664, 757], [518, 590], [511, 602], [65, 298], [259, 665], [12, 590], [529, 582], [291, 589], [187, 660], [296, 526], [64, 159], [170, 320], [64, 724], [65, 187], [508, 665], [298, 660], [358, 752], [592, 706], [25, 664], [397, 619], [466, 691], [52, 664], [665, 736], [350, 609], [665, 666], [664, 669], [248, 602], [66, 666], [118, 463], [311, 590], [64, 346], [309, 590], [312, 506], [319, 611], [660, 750], [65, 185], [706, 757], [764, 766], [271, 520], [394, 466], [150, 319], [367, 636], [611, 621], [64, 265], [186, 602], [10, 509], [89, 350], [590, 605], [312, 508], [101, 311], [358, 710], [399, 591], [664, 756], [507, 699], [229, 312], [98, 466], [661, 757], [165, 463], [62, 350], [328, 592], [586, 660], [68, 665], [156, 350], [633, 664], [218, 713], [311, 607], [187, 757], [511, 766], [338, 466], [559, 664], [63, 664], [318, 507], [509, 664], [659, 660], [64, 753], [249, 664], [169, 320], [10, 209], [584, 660], [422, 665], [394, 664], [19, 590], [341, 703], [101, 466], [231, 660], [41, 64], [38, 590], [350, 445], [7, 546], [350, 711], [631, 664], [188, 664], [65, 177], [100, 296], [410, 660], [50, 766], [287, 766], [187, 665], [24, 660], [160, 350], [34, 507], [303, 529], [472, 660], [421, 422], [549, 621], [81, 466], [660, 736], [61, 590], [664, 668], [515, 602], [99, 350], [260, 350], [246, 466], [71, 664], [660, 663], [302, 602], [220, 664], [310, 350], [259, 602], [431, 660], [558, 660], [120, 766], [665, 761], [64, 91], [179, 248], [36, 664], [664, 735], [16, 466], [10, 237], [107, 766], [66, 592], [592, 691], [338, 602], [177, 466], [33, 367], [350, 433], [64, 403], [220, 602], [12, 529], [106, 664], [65, 100], [216, 602], [507, 627], [656, 696], [196, 529], [373, 660], [350, 761], [529, 713], [466, 593], [77, 660], [86, 397], [305, 367], [63, 367], [638, 664], [175, 665], [305, 466], [72, 367], [520, 521], [220, 660], [64, 510], [96, 367], [10, 724], [367, 706], [248, 287], [185, 397], [64, 621], [303, 466], [91, 592], [466, 664], [86, 590], [438, 507], [64, 754], [65, 539], [42, 590], [341, 342], [36, 590], [176, 529], [286, 592], [150, 279], [350, 714], [65, 656], [38, 766], [656, 752], [431, 664], [583, 665], [472, 664], [197, 466], [726, 766], [592, 692], [656, 660], [266, 507], [180, 592], [660, 758], [367, 608], [448, 466], [96, 590], [445, 592], [350, 754], [287, 466], [229, 766], [116, 397], [260, 590], [466, 585], [87, 279], [57, 665], [187, 367], [37, 529], [10, 37], [64, 208], [602, 754], [186, 279], [15, 664], [65, 661], [86, 463], [196, 507], [45, 664], [67, 690], [65, 206], [267, 591], [466, 515], [397, 746], [294, 766], [237, 367], [589, 592], [502, 766], [37, 320], [367, 763], [507, 729], [397, 627], [109, 755], [64, 90], [180, 466], [65, 604], [257, 766], [207, 592], [168, 602], [176, 248], [506, 660], [276, 397], [636, 660], [664, 764], [179, 463], [100, 466], [457, 660], [420, 660], [250, 590], [255, 602], [638, 660], [367, 509], [40, 279], [507, 609], [98, 766], [603, 665], [318, 529], [64, 401], [126, 590], [166, 590], [218, 508], [393, 507], [63, 766], [276, 507], [41, 664], [341, 761], [311, 714], [111, 664], [169, 766], [341, 376], [10, 80], [97, 466], [552, 660], [664, 703], [295, 529], [664, 692], [323, 664], [250, 602], [96, 766], [287, 602], [219, 466], [79, 660], [585, 766], [250, 529], [313, 665], [590, 659], [278, 420], [32, 466], [10, 706], [147, 350], [10, 311], [289, 766], [43, 466], [10, 585], [246, 463], [350, 712], [10, 309], [397, 441], [432, 664], [664, 693], [583, 664], [29, 602], [35, 311], [235, 660], [41, 507], [397, 668], [325, 350], [466, 746], [342, 664], [67, 607], [238, 757], [63, 590], [52, 296], [32, 660], [265, 463], [15, 312], [397, 447], [227, 602], [65, 394], [10, 109], [219, 660], [665, 767], [110, 466], [88, 660], [664, 742], [96, 397], [586, 664], [664, 694], [358, 615], [298, 590], [279, 296], [65, 750], [65, 101], [185, 311], [64, 539], [52, 288], [72, 664], [366, 664], [218, 311], [117, 664], [326, 766], [466, 590], [592, 754], [54, 664], [226, 529], [65, 447], [74, 665], [507, 753], [279, 744], [219, 592], [67, 529], [91, 602], [111, 367], [664, 738], [279, 316], [249, 665], [65, 757], [350, 763], [602, 714], [305, 660], [8, 367], [110, 367], [8, 375], [664, 736], [15, 602], [137, 665], [65, 313], [219, 507], [345, 664], [312, 764], [236, 248], [116, 664], [350, 651], [32, 218], [664, 746], [302, 592], [220, 279], [99, 766], [195, 664], [278, 664], [265, 466], [227, 529], [650, 664], [206, 466], [296, 589], [218, 583], [664, 700], [279, 286], [62, 367], [248, 367], [64, 78], [209, 507], [218, 660], [279, 314], [518, 664], [10, 633], [515, 590], [139, 397], [507, 668], [666, 741], [64, 294], [259, 660], [350, 518], [2, 507], [341, 737], [394, 660], [65, 763], [10, 439], [150, 312], [250, 311], [41, 65], [139, 350], [64, 629], [66, 549], [313, 529], [118, 766], [401, 660], [107, 350], [167, 590], [64, 176], [180, 350], [67, 279], [262, 660], [660, 704], [15, 64], [67, 590], [240, 660], [466, 539], [77, 397], [466, 765], [350, 407], [297, 766], [127, 367], [306, 660], [100, 320], [9, 350], [47, 507], [338, 592], [53, 592], [367, 712], [51, 665], [65, 226], [91, 664], [10, 371], [65, 765], [10, 299], [65, 139], [438, 664], [230, 466], [367, 760], [62, 319], [155, 320], [33, 664], [121, 466], [260, 320], [602, 755], [327, 664], [582, 755], [549, 691], [16, 296], [508, 660], [403, 666], [35, 64], [16, 602], [108, 466], [238, 350], [88, 602], [590, 666], [65, 744], [64, 611], [240, 529], [320, 606], [107, 529], [159, 466], [177, 665], [186, 529], [108, 592], [664, 712], [52, 592], [515, 660], [47, 311], [197, 633], [425, 660], [396, 529], [150, 660]]
feature_pair_plus_list2=[[466, 529], [664, 759], [602, 766], [64, 665], [279, 590], [397, 592], [311, 621], [248, 755], [660, 768], [218, 666], [65, 278], [549, 607], [16, 402], [53, 757], [463, 526], [197, 312], [507, 762], [320, 619], [367, 380], [10, 350], [62, 401], [52, 756], [610, 633], [0, 656], [319, 758], [38, 50], [288, 296], [67, 584], [48, 611], [422, 724], [249, 591], [58, 686], [287, 295], [341, 589], [208, 728], [66, 508], [44, 605], [4, 358], [9, 695], [54, 99], [100, 258], [425, 577], [3, 655], [581, 765], [8, 694], [519, 658], [267, 582], [587, 706], [290, 420], [717, 730], [282, 561], [177, 238], [36, 326], [727, 763], [423, 543], [599, 631], [283, 657], [516, 689], [235, 764], [109, 187], [291, 608], [137, 767], [51, 292], [214, 721], [293, 346], [200, 268], [157, 289], [219, 229], [623, 653], [281, 533], [570, 732], [138, 745], [314, 667], [663, 761], [315, 421], [375, 400], [565, 722], [228, 305], [514, 718], [79, 176], [316, 510], [280, 317], [212, 720], [652, 671], [110, 441], [313, 403], [41, 744], [284, 465], [251, 646], [175, 632], [209, 743], [179, 196], [390, 712], [364, 622], [28, 723], [506, 650], [37, 195], [571, 683], [211, 426], [42, 170], [132, 677], [640, 742], [71, 643], [662, 737], [472, 736], [682, 705], [265, 559], [5, 332], [525, 680], [536, 678], [546, 675], [69, 342], [15, 318], [56, 255], [556, 673], [641, 719], [545, 716], [700, 729], [298, 609], [669, 735], [299, 410], [80, 239], [459, 734], [300, 354], [141, 749], [440, 731], [377, 513], [107, 121], [351, 419], [140, 359], [368, 439], [376, 509], [372, 414], [381, 424], [33, 217], [405, 518], [636, 760], [46, 150], [461, 578], [216, 257], [39, 445], [55, 139], [49, 399], [620, 634], [149, 156], [74, 699], [133, 547], [604, 740], [460, 687], [189, 386], [207, 615], [90, 748], [353, 703], [17, 259], [418, 512], [178, 739], [432, 698], [301, 747], [322, 404], [352, 535], [59, 323], [324, 407], [325, 408], [417, 464], [523, 684], [455, 738], [435, 676], [87, 190], [601, 701], [76, 681], [345, 702], [45, 186], [406, 752], [447, 704], [237, 697], [167, 751], [159, 696], [369, 453], [148, 693], [411, 733], [285, 365], [81, 588], [467, 616], [603, 709], [206, 708], [710, 741], [146, 327], [348, 638], [247, 711], [134, 524], [647, 679], [443, 674], [451, 558], [271, 378], [412, 434], [310, 337], [450, 541], [449, 542], [82, 231], [83, 232], [84, 233], [85, 234], [135, 335], [213, 437], [477, 566], [476, 575], [7, 112], [478, 552], [113, 261], [114, 262], [115, 263], [241, 457], [361, 574], [427, 528], [111, 128], [488, 548], [360, 520], [242, 494], [484, 562], [343, 521], [252, 487], [253, 486], [92, 254], [356, 642], [553, 624], [93, 264], [94, 243], [95, 244], [415, 557], [88, 302], [468, 551], [215, 502], [198, 391], [446, 544], [362, 563], [585, 726], [495, 560], [573, 725], [30, 245], [452, 564], [370, 572], [474, 568], [339, 645], [379, 555], [483, 567], [23, 205], [522, 554], [538, 654], [473, 576], [75, 413], [155, 168], [504, 531], [498, 532], [166, 180], [344, 480], [493, 537], [122, 221], [123, 222], [124, 223], [125, 224], [534, 600], [492, 651], [236, 260], [19, 127], [57, 392], [485, 569], [31, 586], [86, 108], [14, 511], [505, 550], [225, 595], [330, 442], [202, 456], [61, 63], [126, 246], [6, 329], [458, 540], [47, 396], [306, 659], [334, 475], [21, 644], [490, 648], [89, 750], [527, 685], [43, 627], [479, 707], [12, 433], [158, 227], [60, 393], [444, 625], [294, 753], [366, 672], [18, 185], [371, 628], [596, 626], [389, 470], [11, 169], [210, 307], [188, 308], [309, 328], [40, 539], [203, 688], [22, 454], [374, 481], [1, 491], [438, 501], [436, 639], [387, 530], [431, 598], [129, 489], [349, 373], [469, 597], [70, 462], [72, 471], [333, 649], [276, 338], [68, 226], [32, 713], [409, 496], [429, 497], [102, 304], [103, 256], [104, 118], [105, 355], [266, 515], [204, 416], [106, 606], [363, 428], [77, 382], [27, 171], [29, 635], [25, 172], [277, 448], [173, 357], [117, 250], [26, 174], [331, 499], [73, 637], [230, 240], [272, 503], [116, 160], [142, 269], [2, 78], [145, 500], [714, 754], [161, 340]]
feature_pair_sub_list2=[[520, 521], [271, 521], [271, 520], [67, 466], [623, 664], [7, 536], [66, 529], [531, 532], [561, 562], [248, 602], [570, 571], [218, 766], [64, 765], [208, 590], [423, 660], [312, 463], [290, 592], [621, 755], [52, 311], [65, 422], [350, 656], [278, 420], [320, 633], [507, 761], [0, 341], [139, 665], [10, 724], [53, 319], [367, 698], [279, 421], [9, 358], [48, 287], [375, 653], [397, 728], [197, 666], [38, 295], [402, 758], [403, 757], [549, 584], [238, 258], [296, 526], [586, 607], [291, 591], [62, 289], [16, 288], [581, 589], [8, 380], [655, 683], [58, 582], [54, 587], [377, 657], [44, 81], [292, 735], [610, 706], [177, 401], [109, 249], [187, 756], [425, 768], [293, 723], [61, 631], [365, 730], [378, 658], [100, 239], [37, 47], [12, 632], [313, 318], [541, 542], [268, 718], [36, 46], [28, 700], [306, 763], [79, 611], [361, 722], [727, 762], [11, 506], [588, 686], [99, 267], [360, 721], [717, 759], [431, 432], [679, 720], [508, 605], [282, 391], [285, 745], [23, 519], [445, 599], [379, 716], [250, 255], [176, 326], [4, 314], [348, 671], [110, 229], [166, 167], [15, 29], [230, 235], [585, 764], [198, 283], [157, 650], [219, 767], [307, 744], [308, 315], [309, 316], [400, 743], [496, 497], [455, 472], [317, 669], [413, 689], [87, 128], [196, 327], [281, 356], [488, 498], [92, 102], [93, 103], [94, 104], [95, 105], [410, 736], [257, 266], [373, 652], [642, 732], [228, 441], [17, 137], [42, 179], [704, 712], [719, 751], [107, 305], [408, 622], [88, 118], [731, 752], [220, 265], [3, 200], [71, 339], [56, 178], [82, 484], [494, 504], [212, 516], [342, 697], [351, 404], [353, 370], [13, 132], [337, 619], [628, 742], [359, 702], [368, 709], [651, 737], [376, 708], [414, 638], [424, 710], [240, 245], [33, 406], [405, 693], [637, 760], [385, 614], [39, 73], [55, 711], [49, 407], [286, 634], [346, 371], [389, 618], [160, 195], [259, 749], [45, 59], [645, 646], [502, 699], [40, 635], [492, 695], [347, 740], [274, 464], [111, 175], [215, 386], [149, 190], [615, 629], [272, 461], [146, 748], [449, 450], [338, 703], [430, 739], [310, 747], [459, 499], [18, 19], [277, 694], [476, 477], [148, 150], [302, 322], [323, 382], [14, 324], [57, 325], [517, 738], [43, 701], [336, 418], [465, 489], [138, 276], [83, 468], [448, 696], [205, 426], [294, 298], [299, 753], [300, 511], [127, 301], [144, 214], [204, 467], [608, 609], [170, 741], [143, 734], [133, 398], [191, 675], [192, 680], [269, 284], [237, 256], [80, 155], [193, 678], [194, 673], [130, 676], [21, 543], [134, 202], [142, 681], [662, 663], [363, 667], [135, 273], [331, 332], [674, 677], [112, 474], [199, 682], [275, 412], [168, 217], [345, 362], [460, 470], [171, 563], [490, 500], [247, 559], [172, 548], [366, 558], [173, 231], [174, 232], [270, 411], [181, 514], [161, 233], [162, 234], [169, 189], [343, 352], [211, 213], [84, 486], [344, 687], [85, 487], [182, 369], [163, 568], [164, 572], [512, 513], [183, 577], [349, 533], [612, 684], [184, 525], [151, 546], [152, 556], [153, 566], [154, 575], [27, 553], [446, 705], [384, 613], [515, 729], [5, 644], [25, 451], [86, 225], [241, 357], [221, 251], [222, 252], [223, 253], [224, 254], [493, 503], [552, 672], [26, 242], [180, 216], [523, 535], [243, 340], [24, 244], [261, 538], [89, 90], [262, 374], [263, 630], [264, 715], [428, 551], [117, 156], [201, 524], [51, 433], [60, 419], [522, 528], [647, 670], [627, 733], [131, 544], [534, 564], [434, 596], [545, 555], [78, 573], [77, 554], [185, 364], [136, 520], [495, 505], [435, 597], [2, 390], [113, 457], [120, 478], [114, 594], [115, 321], [603, 604], [260, 565], [303, 574], [547, 595], [209, 392], [396, 557], [537, 567], [576, 639], [41, 560], [165, 518], [297, 550], [96, 121], [122, 482], [119, 123], [91, 124], [116, 125], [437, 600], [569, 578], [30, 415], [746, 750], [76, 443], [393, 643], [453, 469], [456, 483], [442, 725], [74, 438], [203, 527], [479, 480], [98, 540], [458, 485], [141, 661], [31, 236], [394, 620], [32, 640], [473, 475], [34, 354], [452, 462], [713, 714], [72, 330], [659, 726], [580, 648], [75, 707], [35, 427], [329, 754], [97, 108], [186, 668], [395, 399], [6, 471], [246, 690], [510, 593], [101, 444], [439, 691], [20, 688], [454, 692], [106, 481], [147, 491], [126, 501], [436, 583], [416, 530], [226, 227], [598, 606], [383, 685], [50, 69], [509, 539], [328, 624], [68, 654], [158, 159], [63, 641], [304, 372], [355, 649], [280, 636], [188, 333], [625, 626], [1, 417], [206, 440], [22, 387], [129, 429], [334, 381], [70, 616], [145, 335], [140, 210]]
#loss_feature = ['f2','diff_f528_f274', 'diff_f528_f527','diff_f8_f532', 'diff_f8_f543','f332','f67','f25','f120','f766','f376','f39','f670','f228','f652','f415','f596','f406','f13','f355']
#ten_feature = ['f766', 'f597', 'f91', 'diff_f274_f527', 'f334', 'f263', 'f377', 'f710', 'f596', 'f526']
#ten_feature = ['diff_f528_f527', 'diff_f274_f527', 'f2', 'f271', 'diff_f8_f582', 'f222', 'diff_f274_f528', 'f332', 'f576', 'f670']
for i in range(len(features[:100])):
    loss_feature.append(test.columns[features[i]])
    
rep = []
for i in feature_pair_sub_list_sf:
    if i not in rep:
        rep.append(i)
feature_pair_sub_list_sf = rep
rep = []
for i in feature_pair_plus_list2:
    if i not in rep:
        rep.append(i)
feature_pair_plus_list2 = rep

for i in range(len(feature_pair_sub_list_sf)):
    feature1 = feature_pair_sub_list_sf[i][0]
    feature2 = feature_pair_sub_list_sf[i][1]
    feature = str(feature1)+"-"+str(feature2)
    loss_feature.append(feature)
    test[feature] = test.iloc[:,feature1] - test.iloc[:,feature2]
    print("feature {} created.".format(feature))
for i in range(len(feature_pair_plus_list2[:100])):
    feature1 = feature_pair_plus_list2[i][0]
    feature2 = feature_pair_plus_list2[i][1]
    feature = str(feature1)+"+"+str(feature2)
    loss_feature.append(feature)
    test[feature] = test.iloc[:,feature1] + test.iloc[:,feature2]
    print("feature {} created.".format(feature))
for i in range(len(feature_pair_mul_list[:40])):
    feature1 = feature_pair_mul_list[i][0]
    feature2 = feature_pair_mul_list[i][1]
    feature = str(feature1)+"*"+str(feature2)
    loss_feature.append(feature)
    test[feature] = test.iloc[:,feature1] * test.iloc[:,feature2]
    print("feature {} created.".format(feature))
'''for i in range(len(feature_pair_divide_list)):
    feature1 = feature_pair_divide_list[i][0]
    feature2 = feature_pair_divide_list[i][1]
    if (all(v != 0 for v in test.iloc[:,feature2].values)):
        value = test.iloc[:,feature1] / test.iloc[:,feature2]
        feature = str(feature1)+"/"+str(feature2)
        test[feature] = value
        loss_feature.append(feature)
        print("feature {} created.".format(feature))
for i in range(len(feature_pair_sub_mul_list)):
    feature1 = feature_pair_sub_mul_list[i][0]
    feature2 = feature_pair_sub_mul_list[i][1]
    feature3 = feature_pair_sub_mul_list[i][2]
    feature = str(feature1)+"-"+str(feature2)+"*"+str(feature3)
    loss_feature.append(feature)
    test[feature] = (test.iloc[:,feature1] - test.iloc[:,feature2]) * test.iloc[:,feature3]
    print("feature {} created.".format(feature))'''

feature 271-521 created.
feature 520-521 created.
feature 271-520 created.
feature 592-608 created.
feature 238-248 created.
feature 186-766 created.
feature 60-311 created.
feature 176-218 created.
feature 9-397 created.
feature 248-750 created.
feature 90-529 created.
feature 396-766 created.
feature 660-724 created.
feature 108-507 created.
feature 107-660 created.
feature 65-320 created.
feature 577-660 created.
feature 10-455 created.
feature 64-126 created.
feature 56-312 created.
feature 664-757 created.
feature 518-590 created.
feature 511-602 created.
feature 65-298 created.
feature 259-665 created.
feature 12-590 created.
feature 529-582 created.
feature 291-589 created.
feature 187-660 created.
feature 296-526 created.
feature 64-159 created.
feature 170-320 created.
feature 64-724 created.
feature 65-187 created.
feature 508-665 created.
feature 298-660 created.
feature 358-752 created.
feature 592-706 created.
feature 25-664 created.
feature 397-619 created.
feature 466-69

feature 79+176 created.
feature 316+510 created.
feature 280+317 created.
feature 212+720 created.
feature 652+671 created.
feature 110+441 created.
feature 313+403 created.
feature 41+744 created.
feature 284+465 created.
feature 251+646 created.
feature 175+632 created.
feature 209+743 created.
feature 179+196 created.
feature 390+712 created.
feature 364+622 created.
feature 28+723 created.
feature 506+650 created.
feature 37+195 created.
feature 571+683 created.
feature 211+426 created.
feature 42+170 created.
feature 466*529 created.
feature 621*664 created.
feature 159*626 created.
feature 599*602 created.
feature 213*607 created.
feature 209*218 created.
feature 433*463 created.
feature 16*665 created.
feature 619*766 created.
feature 158*625 created.
feature 558*605 created.
feature 64*248 created.
feature 402*660 created.
feature 583*606 created.
feature 53*279 created.
feature 595*596 created.
feature 367*590 created.
feature 592*633 created.
feature 52*278 created.
feature 6

'for i in range(len(feature_pair_divide_list)):\n    feature1 = feature_pair_divide_list[i][0]\n    feature2 = feature_pair_divide_list[i][1]\n    if (all(v != 0 for v in test.iloc[:,feature2].values)):\n        value = test.iloc[:,feature1] / test.iloc[:,feature2]\n        feature = str(feature1)+"/"+str(feature2)\n        test[feature] = value\n        loss_feature.append(feature)\n        print("feature {} created.".format(feature))\nfor i in range(len(feature_pair_sub_mul_list)):\n    feature1 = feature_pair_sub_mul_list[i][0]\n    feature2 = feature_pair_sub_mul_list[i][1]\n    feature3 = feature_pair_sub_mul_list[i][2]\n    feature = str(feature1)+"-"+str(feature2)+"*"+str(feature3)\n    loss_feature.append(feature)\n    test[feature] = (test.iloc[:,feature1] - test.iloc[:,feature2]) * test.iloc[:,feature3]\n    print("feature {} created.".format(feature))'

In [29]:
len(loss_feature)

740

In [9]:
test

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,497-660*501,102-660*263,103-660*264,104-660*75,105-660*434,724-727*642,476-660*636,112-660*354,457-660*498,113-660*363
0,-0.382530,0.445183,-0.887742,-0.985801,1.673296,0.990708,6.384724e-01,0.712434,-0.512582,-0.408762,...,1.291944e+00,1.396969,1.396969,1.534289e+00,1.577004,0.062751,1.181178e+00,1.507012e+00,7.071165e-01,6.402632e-01
1,0.636094,0.445183,1.470515,0.656789,-0.846063,0.988634,-9.251128e-01,-0.714153,0.714351,0.670968,...,6.467467e-01,0.587349,0.587349,6.549316e-01,0.655647,0.186232,-7.454395e-01,5.070048e-01,5.258984e-01,5.841741e-01
2,1.654719,-0.741023,-0.568070,2.013712,-0.652266,0.802070,6.338773e-02,0.707979,1.295986,1.599701,...,3.685941e-03,0.051680,0.051680,-1.755638e-04,0.005974,-0.152553,-9.051383e-02,-1.467435e-01,3.114670e-03,-1.549000e-01
3,-1.197430,-0.147920,1.626338,-0.985801,-0.652266,0.878747,-1.092289e+00,1.099154,-1.181752,-1.195462,...,-4.174991e-01,0.386495,0.386495,-4.284911e-01,-0.420344,0.001887,1.039509e-01,4.261021e-01,-5.888983e-01,1.810321e-01
4,0.364461,-1.334125,-0.102379,-0.057381,-0.070875,1.009037,-1.788770e-16,2.927609,-0.097335,0.292442,...,-1.658191e-17,0.334443,0.334443,2.155566e-17,0.000000,-0.196763,8.013670e-02,-3.220775e-02,-1.709155e-17,-2.566845e-02
5,-0.518347,1.038286,-1.581170,-0.985801,1.673296,1.040173,1.743799e+00,0.572538,-0.731057,-0.570101,...,2.895241e+00,0.078498,0.078498,3.163442e+00,3.129123,-0.172639,-1.564621e+00,-1.592630e+00,1.508928e+00,-1.254085e+00
6,-0.314622,-0.741023,-0.888624,-0.985801,-0.652266,0.812691,1.405514e+00,3.407445,-0.281809,-0.298445,...,1.318077e+00,3.096476,3.096476,1.338245e+00,1.271083,0.346282,-7.155514e-01,-6.803884e-01,1.016465e+00,-6.369381e-01
7,0.160736,-0.147920,-1.198937,-0.343049,-0.652266,-0.972438,-5.136637e-01,0.851439,0.083522,0.172472,...,7.850751e-02,-0.073352,-0.073352,7.430100e-02,0.070548,0.209909,-3.169106e-01,-7.648169e-03,1.272199e-01,3.551836e-03
8,1.179361,0.445183,-1.564569,0.656789,1.091906,0.899738,-5.852543e-01,-0.608562,1.413905,1.214281,...,-2.322977e-01,-0.205545,-0.205545,-2.314955e-01,-0.245022,-0.032895,5.401870e-01,-1.314568e-01,-2.480781e-01,-2.151178e-01
9,1.654719,-0.741023,-1.190792,0.728206,-0.652266,-1.079915,1.739866e+00,0.435760,1.809620,1.703813,...,2.557958e+00,1.423686,1.423686,1.473212e+00,1.209624,-0.030818,-1.520673e+00,-2.936681e+00,9.541329e+00,-3.166741e+00


In [10]:
test.loc[:,'default'] = test['loss'].apply(lambda x: 1 if x > 0 else 0)

In [11]:
xgb_feature = ['f2','diff_f528_f274', 'diff_f528_f527','diff_f8_f532', 'diff_f8_f543','f332','f67','f25','f120','f766','f376','f39','f670','f228','f652','f415','f596','f406','f13','f355']
for feature in classify_feature:
    if len(feature.split("_")) == 3:
            feature1, feature2 = feature.split("_")[1:]
            test[feature] = test[feature1] - test[feature2]
            print("feature {} created.".format(feature))
for feature in xgb_feature:
    if len(feature.split("_")) == 3:
            feature1, feature2 = feature.split("_")[1:]
            test[feature] = test[feature1] - test[feature2]
            print("feature {} created.".format(feature))

feature diff_f528_f527 created.
feature diff_f274_f527 created.
feature diff_f8_f582 created.
feature diff_f274_f528 created.
feature diff_f528_f274 created.
feature diff_f528_f527 created.
feature diff_f8_f532 created.
feature diff_f8_f543 created.


In [70]:
#xgb_classifier = XGBClassifier(max_depth=6, silent=False, n_jobs=2, reg_lambda=1, n_estimators=100, random_state=19)
#nn_classifier = MLPClassifier(hidden_layer_sizes=(64), alpha=1, max_iter=1000, verbose=True)

print('Voting Classifier Training')
kf = KFold(n_splits=10)
kf.get_n_splits(test)
total = 0
#for train_index, test_index in kf.split(test):
#    print("TRAIN:", train_index, "TEST:", test_index)
#    class_index = train_index[:22500]
#    regress_index = train_index[22500:]
xgb_classifier = XGBClassifier(max_depth=6, silent=False, n_jobs=2, reg_lambda=1, n_estimators=100, random_state=19)

#gb_classifier = GradientBoostingClassifier(max_depth=5, verbose=1, n_estimators=100, random_state=17)
xgb_regressor = XGBRegressor(max_depth=8, silent=False, n_jobs=2, reg_lambda=1, n_estimators=40, random_state=19,learning_rate=0.09)
#bigger_regressor = XGBRegressor(max_depth=8, silent=False, n_jobs=2, reg_lambda=1, n_estimators=40, random_state=19,learning_rate=0.09)
svr = SVR(C=16, kernel='rbf', gamma =  0.000122)
#rf = RandomForestRegressor(n_estimators=275,max_depth=22, min_samples_split=2,max_features='sqrt')
print("done init")
#votingClassifier = VotingClassifier(estimators=[('svm', svm), ('rf', rf_classifier), ('gb', gb_classifier), ('nn', nn_classifier), ('xgb', xgb_classifier)], voting='hard', n_jobs=-1)
#    X_train_class, X_train_regress, X_test  = test.iloc[class_index], test.iloc[regress_index], test.iloc[test_index]
ind_train = np.where(test['loss'] > 0)[0]
regress_train = test.iloc[ind_train,:]
print("start fitting")
#print(regress_train[loss_feature])
#break
#rf.fit(regress_train[loss_feature], regress_train['loss'])
xgb_classifier.fit(test[classify_feature], test['default'])
#print(np.isfinite(regress_train[loss_feature]).all())
xgb_regressor.fit(test[xgb_feature], test['loss'])
svr.fit(regress_train[loss_feature], regress_train['loss'])


Voting Classifier Training
done init
start fitting


SVR(C=16, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.000122,
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [71]:
with open("votingmodel.pkl", "wb") as votingpkl:
    pickle.dump(xgb_classifier, votingpkl)
with open("svr.pkl", "wb") as svrpkl:
    pickle.dump(svr, svrpkl)
with open("xgb.pkl", "wb") as xgbpkl:
    pickle.dump(xgb_regressor, xgbpkl)
class_predict = xgb_classifier.predict(X_test[classify_feature])
svr_predict = svr.predict(X_test[loss_feature])
xgb_predict = xgb_regressor.predict(X_test[xgb_feature])
#rf_predict = rf.predict(X_test[loss_feature])
for i in range(len(class_predict)):
    if (class_predict[i] != 0):
        tot = 0.75*svr_predict[i] + 0.25*xgb_predict[i]
        if (tot < 1):
            tot = 0
        class_predict[i] = tot

